⚠️ 이 노트북은 서울 건물연식지도 데이터 전처리 과정을 전반적으로 공유하기 위해서 작성되었습니다. 리포에 원본 데이터를 커밋하지 않았으므로 바로 실행되지 않습니다. 

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.colors as mc
import os
from lonboard.geoarrow.geopandas_interop import geopandas_to_geoarrow
import pyarrow as pa

### Read Building data 2017/2024

In [ ]:
b_2023_origin = gpd.read_file('./F_FAC_BUILDING_서울.zip', engine="pyogrio", encoding='euc-kr')

In [ ]:
b_2023_origin.shape

In [ ]:
b_2023 = b_2023_origin.copy()
b_2023 = b_2023.to_crs(4326)

#### Filter out outliars manually

In [ ]:
b_2023 = b_2023[b_2023.UFID !='2020202526764540434800000000']
b_2023 = b_2023[b_2023.UFID !='2019202463904539575400000000']

#### Give arbiturary value to the buildings with unlikely HEIGHT, invalid USEAPR_DAY values

In [ ]:
b_2023.loc[b_2023['HEIGHT'] > 10000, 'HEIGHT'] = 5

b_2023['APR_Y'] = b_2023['USEAPR_DAY'].str[:4]
b_2023['APR_Y'] = b_2023['APR_Y'].astype(float)
b_2023.loc[b_2023['APR_Y'] < 1000, 'APR_Y'] = None
b_2023.loc[b_2023['APR_Y'] > 2024, 'APR_Y'] = None

In [ ]:
b_2017_origin = gpd.read_file('./2017-data.geojson', engine="pyogrio")
b_2017 = b_2017_origin.copy()

In [ ]:
b_2017.head()

#### Read helper data

In [ ]:
# File from https://www.vworld.kr/dtmk/dtmk_ntads_s002.do?dsId=30603
dong_origin = gpd.read_file('./LSMD_ADM_SECT_UMD_서울.zip', engine='pyogrio', encoding='euc-kr') 
dong = dong_origin[['EMD_CD','EMD_NM', 'geometry']]
dong = dong.to_crs('4326')

In [ ]:
dong.head()

#### Assign 법정동 to buildings

In [ ]:
b_2023['COL_ADM_SE'].head(3)

In [ ]:
dong['bd_code'] = dong['EMD_CD'].astype(str).str[:8]
b_2023['bd_code'] = b_2023['BD_MGT_SN'].astype(str).str[:8]
b_2023['BEONJI'] = b_2023['BD_MGT_SN'].str[11:19]

bw_dong = pd.merge(b_2023, dong, on='bd_code', how='left')
bw_dong['DONG'] = bw_dong['EMD_NM']

bw_dong['APR_Y'] = bw_dong['APR_Y'].astype(float)
bw_dong = bw_dong.rename({'geometry_x': 'geometry'}, axis=1)

bw_dong_s = gpd.GeoDataFrame(bw_dong[['DONG','BEONJI','APR_Y', 'USEAPR_DAY', 'HEIGHT', 'BLD_NM', 'geometry']])

In [ ]:
# Check if the data was processed as expected: 석탄회관 - 수송동 80-6
bw_dong_s[bw_dong_s.BLD_NM =='석탄회관']

In [ ]:
bw_dong_s[bw_dong_s.BLD_NM =='영성빌라'] # 오류동 1-17

In [ ]:
b_2017['bd_code'] = b_2017['COL_ADM_SE'].astype(str).str[:8]
b_2017['APR_Y'] = b_2017['USEAPR_DAY'].str[:4]
b_2017['APR_Y'] = b_2017['APR_Y'].astype(float)
bw_dong_2017 = pd.merge(b_2017, dong, on='bd_code', how='left')
bw_dong_2017 = bw_dong_2017.rename({'geometry_x': 'geometry'}, axis=1)

In [ ]:
bw_dong_2017['DONG'] = bw_dong_2017['EMD_NM']
bw_dong_2017 = bw_dong_2017.rename({'geometry_x':'geometry'}, axis=1)

In [ ]:
bw_dong_2017_s = gpd.GeoDataFrame(bw_dong_2017[['DONG','BEONJI','APR_Y', 'USEAPR_DAY', 'geometry']])

#### Centroids

In [ ]:
bc_2023 = b_2023.copy()
centroids_2023 = b_2023['geometry'].centroid
bc_2023['geometry'] = centroids_2023

bc_2017 = b_2017.copy()
centroids_2017 = b_2017['geometry'].centroid
bc_2017['geometry'] = centroids_2017

#### Get rid of null data from centroids

In [ ]:
bc_2017 = bc_2017[~bc_2017['APR_Y'].isna()]
bc_2023 = bc_2023[~bc_2023['APR_Y'].isna()]
bc_2023 = bc_2023[['APR_Y', 'geometry']]
bc_2017 = bc_2017[['APR_Y', 'geometry']]


In [ ]:
bc_2023.head(3)

### Write processed files

In [ ]:
file_path_2023 = './output/bd_2023.geojson'
file_path_2017 = './output/bd_2017.geojson'

file_path_2023c = './output/bdc_2023.geojson'
file_path_2017c = './output/bdc_2017.geojson'

bw_dong_s.to_file(file_path_2023)
bw_dong_2017_s.to_file(file_path_2017)
bc_2017.to_file(file_path_2017c)
bc_2023.to_file(file_path_2023c)

#### Group with 법정동

In [ ]:
dong['DONG'] = dong['EMD_NM']

In [ ]:
sample = bw_dong_s[bw_dong_s['DONG']=='사간동']
# sample.to_csv('./sagandong.csv')
sample['APR_Y'].mean()

In [ ]:

jg_2023 = bw_dong_s.groupby(['DONG']).agg(
  APR_Y=('APR_Y', lambda x: np.mean(x) if not x.isna().all() else np.nan),
  numberWData= ('USEAPR_DAY', lambda x: x.notna().sum()), 
  numberWOData= ('USEAPR_DAY', lambda x: x.isna().sum()), 
  )
jg_dong = gpd.GeoDataFrame(dong.merge(jg_2023.reset_index(), how='left'))

In [ ]:
jg_dong[jg_dong['DONG']=='용산동4가']

In [ ]:
jg_dong = jg_dong[['EMD_NM', 'APR_Y', 'numberWData', 'numberWOData', 'geometry']]

In [ ]:
jg_dong.to_file('./output/dong_2023.geojson')

In [ ]:
joined_2017 = gpd.read_file('./final-dong.geojson') # Already aggregated

In [ ]:
joined_2017.columns

In [ ]:
joined_2017['END_NM'] = joined_2017['kr_name']
joined_2017['APR_Y'] = joined_2017['average']
joined_2017 = joined_2017[['END_NM', 'APR_Y', 'numberWData', 'numberWOdata', 'geometry']]
joined_2017.to_file('./output/dong_2017.geojson')


#### Generate pmtiles (Prerequisite: [Tippecanoe](https://github.com/felt/tippecanoe) )

In [ ]:
command = "tippecanoe -z 16 -Z 14 -d 16 -l bd_2023 -o ./output/bd_2023.pmtiles --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-zoom=16 " + file_path_2023
os.system(command)

In [ ]:
command = "tippecanoe -z 16 -Z 14 -d 16 -l bd_2017 -o ./output/bd_2017.pmtiles --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-zoom=16 " + file_path_2017
os.system(command)

##### PMtiles - Centroid data

In [ ]:
command = "tippecanoe -z 13 -Z 13 --no-feature-limit --no-tile-size-limit -l bdc_2023 -o ./output/bdc_2023.pmtiles " + file_path_2023c
os.system(command)

In [ ]:
command = "tippecanoe -z 13 -Z 13 --no-feature-limit --no-tile-size-limit -l bdc_2017 -o ./output/bdc_2017.pmtiles " + file_path_2017c

os.system(command)

### Bonus: Generate GeoParquet

In [ ]:
gpq_b_2023 = b_2023.copy()

#### Encode building age information as color to arrow

In [ ]:
viridis =  ["#440154", "#482475", "#414287", "#355e8d", "#2a768e", "#218e8d", "#21a585", "#3dbc74", "#70cf57", "#b0dd2f"]
viridis.reverse()
rgb_colors = []
for h in viridis:
  rgb_colors.append([int(m *255) for m in mc.to_rgba(h)]) # rgba / rgb matters?
rgb_colors

In [ ]:
def get_color(row):
  year = row['APR_Y']
  if ((np.isnan(year)) or (year < 1850)):
    return [100,100,100,255]
  cidx = int((year - 1930)/10)
  if cidx > 9:
    cidx = 9
  if cidx < 0:
    cidx = 0;
  return rgb_colors[cidx]

color_cols = gpq_b_2023.apply(get_color, axis=1)

In [ ]:
# Convert the color_column to NumPy array
#gpq_b_2023['color_column'] = gpq_b_2023['color'].apply(np.array, dtype=np.uint8)

color_array = np.stack(color_cols.values) # necessary?

### Write parquet files

In [ ]:
# Use Geopanda's api (WKB geometry)
gpq_b_2023.to_parquet('./output/bd_2023_wkb.parquet')

# Use Lonboard's api (Arrow native geometry)
s_arrow = geopandas_to_geoarrow(gpq_b_2023, preserve_index=False)

s_arrow = s_arrow.append_column(
    "color", pa.FixedSizeListArray.from_arrays(color_array.flatten('C'), 4)
)

In [ ]:
pa.parquet.write_table(s_arrow, './output/bd_2023.parquet')